# Progressive Rollouts with Seldon Deplyment AB Tests

In this example we will AB Test two Iris models: an SKLearn model and an XGBOOST model.
We will run a progressive rollout allowing Iter8 to control the traffic to the two Seldon Deployments and gradually move traffic to the best model.

## Install Depenendcies

  * Istio
  * Seldon Core
  * Seldon Core Analytics
  * Iter8
  
  You can run `../../platformsetup.sh` to install dependencies.

## Create ABTest with Two Predictors

In [ ]:
!cat abtest.yaml

In [ ]:
!kubectl apply -f abtest.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=600s pods --all -n ns-production

## Create some load on models.

We will send reqeusts which will be split by the Seldon AB Test as well as random feedback to both models with feedback favouring the candidate

In [ ]:
!cat fortio.yaml

In [ ]:
!URL_VALUE="http://$(kubectl -n istio-system get svc istio-ingressgateway -o jsonpath='{.spec.clusterIP}')" && \
  sed "s+URL_VALUE+${URL_VALUE}+g" fortio.yaml | kubectl apply -f -

In [ ]:
!kubectl wait --for condition=ready --timeout=600s pods --all -n default

## Create Metrics to evaluate 

These are a standard set of metrics we use in all examples.

In [ ]:
!cat ../../metrics.yaml

In [ ]:
!kubectl create -f ../../metrics.yaml

In [ ]:
!kubectl get metrics -n iter8-seldon

## Create Progressive Rollout Experiment

  * Run 15 iterations with 5 second gaps between default and candidate models
  * Both models must pass objectives
  * winnder will be chosen based on user engagement metric

In [ ]:
!cat experiment.yaml

In [ ]:
!kubectl create -f experiment.yaml

## Monitor Experiment

Download iter8ctl. 

```
GO111MODULE=on GOBIN=/usr/local/bin go get github.com/iter8-tools/iter8ctl@v0.1.3
```

Then:

```
while clear; do kubectl get experiment quickstart-exp -o yaml | iter8ctl describe -f -; sleep 8; done
```

By the end you should see the xgboost candidate model is promoted.

In [ ]:
!kubectl wait experiment quickstart-exp --for=condition=Completed --timeout=300s

In [ ]:
!kubectl get experiment quickstart-exp -o yaml

## Cleanup

In [ ]:
!kubectl delete -f fortio.yaml
!kubectl delete -f experiment.yaml
!kubectl delete -f ../../metrics.yaml
!kubectl delete -f abtest.yaml